# General

## Dataset
https://www.kaggle.com/datasets/blastchar/telco-customer-churn

# TODO:
### David:
- pre-processing
  - feature scaling (standardize or normalize ?)
  - mapping
- feature selection
- performance visualization


In [22]:
# Environment and sklearn package must be updated to handle at least sklearn version 1.0 for RocCurveDisplay to work
# Using anaconda CMD.exe Prompt run "conda update --all" to update
import sklearn
print(sklearn.__version__)
del sklearn

1.0.2


### Ideas for the final report.  What is the goal of our final project?

Primary

* Make RandomForestClassifier work and figure out how to optimize its predictive performance.
* Show that it performs better than other algorithms.

Secondary

* Focus it more on understanding why an ML algorithm performs better
* (Overfitting, high correlation, other pitfalls of ML algorithms)?

In [23]:
## Imports & Settings
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split as split
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as skm

# display all columns and rows:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

from copy import deepcopy

<h1 style='color:orange'>NOTE</h1>

### Breiman (2001) Section 5:

* If, after feature selection, it is found that only a handful of features are actually any good, Breiman seems to suggest combining these features into some additional features for datasets with fewer than 3000 instances.  See paper for implementation.

### Section 5.1:

* Breiman on handling non-binary categorical variables when forming combined features.

In [24]:
## Dataset & Preprocessing

# Dataset
df_original = pd.read_csv('dataset/telco-customer-churn.csv', index_col=0)
df = df_original.copy()

# Label Encoding (converting categorical to numerical)
categorical_columns = [
    'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines','InternetService',
    'OnlineSecurity', 'OnlineBackup','DeviceProtection','TechSupport','StreamingTV',
    'StreamingMovies','Contract','PaperlessBilling','PaymentMethod', 'Churn'
]

# convert and replace categorical columns with numerical data
# encoding is done with categorical labels that sorted alphabetically so df = ['c','z','a'] will always encode to [1,2,0]
for col in categorical_columns:
    df[col] = df[col].astype('category')
    df[col] = df[col].cat.codes

# Dataset where rows with blank TotalCharges ' ' are replaced with 0
df_0 = df.copy()
df_0['TotalCharges'].replace(" ", 0, inplace=True)
df_0['TotalCharges'] = df_0['TotalCharges'].astype(float)

# Dataset where rows with blank TotalCharges ' ' are deleted
df_d = df.copy()
df_d.drop(df_d[df_d.TotalCharges == " "].index, inplace=True)
df_d['TotalCharges'] = df_d['TotalCharges'].astype(float)


# Dataset where rows with blank TotalCharges ' ' are replaced with the mean of TotalCharges column
df_m = df.copy()
total_charges_mean = (df_d['TotalCharges'].mean()) # mean from previously calculated dataset
df_m['TotalCharges'].replace(" ", total_charges_mean, inplace=True)
df_m['TotalCharges'] = df_m['TotalCharges'].astype(float)

### Feature Selection

### Mean decrease in impurity

In [25]:
X = df_m.drop("Churn", axis=1)
y = df_m["Churn"]

forest = RandomForestClassifier(random_state=0)
forest.fit(X, y)

feature_names = X.columns.values.tolist()

importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
forest_importances_mdi = pd.Series(importances, index=feature_names)

print(forest_importances_mdi.sort_values(ascending=False))

fig, ax = plt.subplots()
forest_importances_mdi.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI (Mean decrease in impurity)")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

TotalCharges        0.197131
MonthlyCharges      0.180334
tenure              0.151839
Contract            0.074690
PaymentMethod       0.051708
TechSupport         0.045079
OnlineSecurity      0.044053
gender              0.028285
OnlineBackup        0.026994
InternetService     0.026422
PaperlessBilling    0.025286
Partner             0.023297
DeviceProtection    0.023238
MultipleLines       0.022711
SeniorCitizen       0.020900
Dependents          0.019474
StreamingMovies     0.017063
StreamingTV         0.016596
PhoneService        0.004899
dtype: float64
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/daviddada/opt/anaconda3/lib/python3.9/site-packages/IPython/core/formatters.py", line 222, in catch_format_error
    """show traceback on failed format call"""
  File "/Users/daviddada/opt/anaconda3/lib/python3.9/site-packages/IPython/core/formatters.py", line 339, in __call__
    pass
  File "/Users/daviddada/opt/anaconda3/lib/python3.9/site-packages/IPython/core/pylabtools.py", line 151, in print_figure
    fig.canvas.print_figure(bytes_io, **kw)
  File "/Users/daviddada/opt/anaconda3/lib/python3.9/site-packages/matplotlib/backend_bases.py", line 2319, in print_figure
    result = print_method(
  File "/Users/daviddada/opt/anaconda3/lib/python3.9/site-packages/matplotlib/backend_bases.py", line 1648, in wrapper
    return func(*args, **kwargs)
  File "/Users/daviddada/opt/anaconda3/lib/python3.9/site-packages/matplotlib/_api/deprecation.py", line 412, in wrapper
    addendum=(addendum + " " + deprecation_addendum) if addendum
  File "

<Figure size 432x288 with 1 Axes>

### Feature permutation

In [26]:
from sklearn.inspection import permutation_importance

result = permutation_importance(
    forest, X, y, n_repeats=10, random_state=42, n_jobs=2
)
forest_importances_pi = pd.Series(result.importances_mean, index=feature_names)
print(forest_importances_pi.sort_values(ascending=False))

fig, ax = plt.subplots()
forest_importances_pi.plot.bar(yerr=result.importances_std, ax=ax)
ax.set_title("Feature importances using permutation")
ax.set_ylabel("Mean accuracy decrease")
fig.tight_layout()
plt.show()

exception calling callback for <Future at 0x7faa1dd64190 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/Users/daviddada/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/Users/daviddada/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 359, in __call__
    if self.parallel._original_iterator is not None:
  File "/Users/daviddada/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 794, in dispatch_next
    against concurrent consumption of the unprotected iterator.
  File "/Users/daviddada/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 861, in dispatch_one_batch
    # No more tasks available in the iterator: tell caller to stop.
  File "/Users/daviddada/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 779, in _dispatch
    cb = BatchCompletionCallBack(dispatch_timestamp, len(batch), self)
 

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/daviddada/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
    for node,mode in to_run:
  File "/var/folders/77/y_32dvyd2xq345ys9v__fvym0000gn/T/ipykernel_69004/642240020.py", line 3, in <cell line: 3>
    result = permutation_importance(
  File "/Users/daviddada/opt/anaconda3/lib/python3.9/site-packages/sklearn/inspection/_permutation_importance.py", line 259, in permutation_importance
    scores = Parallel(n_jobs=n_jobs)(
  File "/Users/daviddada/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 1056, in __call__
    # No need to wait for async callbacks to trigger to
  File "/Users/daviddada/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 935, in retrieve
  File "/Users/daviddada/opt/anaconda3/lib/python3.9/site-packages/joblib/_parallel_backends.py", line 542, in wrap_future_result
    return future.result(timeout=timeout)
  File "/Users/davidda

<h1 style='color:orange'>NOTE</h1>

### Breiman (2001) Section 9:

* If the input dataset is just a combination of many "weak" (low-variance) features, a higher 'max_features' value will perform better so long as the features have low correlation.

### Probst (2018) Section 2.3:

The formula for Area Under the Curve of a Random Forest for binary classification (either 1 or 0) is:

$AUC = \frac{\sum_{i=1}^{n_1} \sum_{j=1}^{n_2} S(\hat{p}_i^*,\hat{p}_j^{**})}{n_1n_2}$

* $n_1 =$ The number of observations whose class is predicted to be 1
* $n_2 =$ The number of observations whose class is predicted to be 0
* $\hat{p}_{i}^*=$ The number of trees that predicted 1 for the $n_1^{th}$ instance divided by the number of trees in the forest
* $\hat{p}_{j}^{**}=$ The number of trees that predicted 1 for the $n_2^{th}$ instance divided by the number of trees in the forest

$S(\hat{p}_i^*,\hat{p}_j^{**}) =$ A function that returns the following:

* If $(\hat{p}_i^* < \hat{p}_j^{**})$, return 0.0
* If $(\hat{p}_i^* = \hat{p}_j^{**})$, return 0.5
* If $(\hat{p}_i^* > \hat{p}_j^{**})$, return 1.0

The larger this value, the better the classifier is.

Unsure if this is what's implemented by sklearn.

In [ ]:
## Model
class RFC:
    '''
    Parameters:
      * X = A pandas.DataFrame of features
      * y = An associative (same index as X) pandas.Series of labels
      * hyperParameters = A dict whose keys correspond to the parameters of
        RandomForestClassifer and whose values are lists of arguments for those
        parameters
    Other notes:
      * Research paper references throughout this class are done in this manner:
         - Breiman (2001) = B
         - Probst  (2018) = P
        Section and subsection numbers will be added after these aliases like
        B2.1 to show that a comment refers to Breiman (2001) Section 2 Subsection 1
    '''
    def __init__(self, hyperParameters={}):
        hyperParams = \
            self.__getDefaultParams__() if len(hyperParameters) == 0 else self.__validateParams__(hyperParameters)
        self.params = self.__getParamCombos__(hyperParams)
        self.classifiers = []
    
    def __getDefaultParams__(self):
        ''' Supplies default parameters to test based on what our source papers
            suggest are generally good.
        '''
        out = {}
        ''' B4
            Suggests selecting more than 1 features to split a node with
            doesn't perform any better than selecting 1 so long as we have a high
            number of trees.
            
            sklearn sets its default 'max_features'='sqrt', so this is added as one of
            our defaults to investigate why.  Did they find that 'sqrt' performs better
            than Breiman's suggested 1?
        '''
        out['max_features'] = [1, 'sqrt']
        ''' P5.3
            Suggests that the biggest performance gain is experienced by the first 100
            trees.  However, lower 'max_samples' values, lower 'max_depth' values, and
            higher 'max_features' values reduce inter-tree correlation and therefore
            increase the number of trees needed to reach convergence.
        '''
        out['max_depth'] = [None, 3]
        out['n_estimators'] = [100, 200]
        ''' B3
            Suggests each bootstrap sample should ideally have 2/3s the length of X.
        '''
        out['max_samples'] = [float(2/3)]
        
        out['oob_score'] = [True]
        out['n_jobs'] = [-1]
        return out

    def __validateParams__(self, params):
        ''' Removes any misspelled or non-existent-in-RandomForestClassifier
            parameter name keys from the dict passed in by the caller.
        '''
        VALID = ['n_estimators', 'criterion', 'max_depth', 'min_samples_split',
                 'min_samples_leaf', 'min_weight_fraction_leaf', 'max_features',
                 'max_leaf_nodes', 'min_impurity_decrease', 'boostrap',
                 'oob_score','n_jobs', 'random_state', 'verbose', 'warm_start',
                 'class_weight', 'ccp_alpha', 'max_samples']
        invalidKeys = []
        validatedParams = deepcopy(params)
        for key in params:
            if key not in VALID:
                validatedParams.pop(key)
                invalidKeys += [key]
        if len(invalidKeys) > 0:
            print("These are not valid RandomForestClassifier parameter names:\n{}\n".format(invalidKeys) + \
                  "They will not be tuned by GridSearchCV\n")
        return validatedParams if len(validatedParams) > 0 else self.__getDefaultParams__()
    
    def __getParamCombos__(self, dictParams):
        # Assemble a dict for each unique combination of hyper parameters.  These dicts can be passed into
        # a function to supply named arguments
        combinations = []
        for param, valList in dictParams.items():
            if len(combinations) > 0:
                for i in range(len(combinations)):
                    base = combinations.pop(0)
                    for val in valList:
                        newCombo = deepcopy(base)
                        newCombo[param] = val
                        combinations.append(newCombo)
            else:
                for val in valList:
                    combinations.append({param: val})
        return combinations
    
    def train(self, trainX, trainY):
        ''' Train an RFC for each unique combination of hyperparameters
        Code references:
          * Passing named arguments to a function with a dict of names,vals
            https://stackoverflow.com/questions/334655/passing-a-dictionary-to-a-function-as-keyword-parameters
        '''
        print("Training {} classifiers...".format(len(self.params)))
        for dictParams in self.params:
            print("\t{}".format(dictParams))
            newRfc = RandomForestClassifier(**dictParams)
            newRfc.fit(trainX, trainY)
            self.classifiers.append(newRfc)
            
    def showRocAuc(self, testX, testY):
        if len(self.classifiers) == 0:
            raise AttributeError("No classifiers have been trained yet.  You must call .train() to create " + \
                                 "some classifiers to display.")

        fig = plt.figure(dpi=300)
        axes = plt.gca()
        for i in range(len(self.classifiers)):
            skm.RocCurveDisplay.from_estimator(
                self.classifiers[i],
                testX,
                testY,
                pos_label=0,
                name="RFC #{}".format(i+1),
                ax=axes)


## Testing non-default hyper parameters
X_train, X_test, y_train, y_test = split(df_m.drop("Churn", axis=1),  df_m["Churn"], test_size=0.1)
rfc = RFC(hyperParameters={
    'max_features': [1],
    'max_depth': [3],
    'n_estimators': [100, 200, 500, 1000],
    'max_samples': [float(2/3)],
    'oob_score': [True],
    'n_jobs': [-1]
})
rfc.train(X_train, y_train)
rfc.showRocAuc(X_test, y_test)

In [ ]:
## Metrics

# Get predictions
y_pred = rfc.classifiers[0].predict(X_test)

# Confusion Matrix
cf = skm.confusion_matrix(y_pred, y_test)
sns.heatmap(cf/np.sum(cf), fmt='.2%', annot=True, cmap='Blues')

# Classification Report
print(rfc.params[0], end="\n\n")
print(skm.classification_report(y_pred, y_test))